
## Goal

The purpose of this notebook is to implement the following business logic:

- Determine the number of spots to allocate to each county based on its area.
- Prioritize spot allocation near park entries, as BI dashboards indicate parks have the highest sales.
- Favor larger parks with more functionalities (e.g., playgrounds, sport fields) as they are more attractive.
- Prefer park entrances that are more accessible, as these are more desirable locations for spot allocation.

In [0]:
%run ../get_user

In [0]:
user_email = spark.sql("SELECT current_user()").collect()[0][0]
username = get_username_from_email(user_email)
print(username)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import expr

In [0]:
# This block calculates the total area of all UK ceremonial counties and allocates a proportional number of locations to each county based on its area.
# It then orders the counties by the number of allocated locations and saves the result as a new silver table in Unity Catalog.

administrative_boundaries = spark.sql(
    f"""
    SELECT 
        b.fid, 
        b.name, 
        ST_Area(b.geometry)::FLOAT AS area, 
        b.geometry, 
        ST_Geohash(ST_Transform(b.geometry, 4326), 5) AS geohash
    FROM geospatial.lookups_geobrix.boundary_line_ceremonial_counties_{username} b
    """
)

total_locations = 1000
uk_area = administrative_boundaries.selectExpr("SUM(area) AS total_area").first().total_area

administrative_boundaries = (
    administrative_boundaries.withColumn(
        "number_of_locations",
        F.round(F.col("area") / uk_area * F.lit(total_locations)).cast("integer")
    )
    .orderBy("number_of_locations", ascending=True)
)

administrative_boundaries.write.mode("overwrite").saveAsTable(
    f"geospatial.lookups_geobrix.boundary_line_ceremonial_counties_silver_{username}"
)

administrative_boundaries.createOrReplaceTempView("administrative_boundaries_vw")
display(administrative_boundaries)

In [0]:
# This block processes greenspace sites by:
# 1. Filtering for relevant greenspace functions ('Play Space', 'Playing Field', 'Public Park Or Garden').
# 2. Identifying and aggregating sites that are spatially covered by a larger greenspace site, combining their functions and counting the number of unique functions. This is an indicator of attrcativeness.
# 3. Assigning area quantiles (20, 40, 60, 80, 100) to each site based on their area, to categorize them by size.
# 4. Creating unified views for both aggregated and non-covered sites, and combining them into a single DataFrame for further analysis.
# 5. Preparing the resulting DataFrame for downstream prioritization and allocation logic by exposing it as a temporary view.

df_greenspaces_bronze = (
    spark.table(f"geospatial.greenspaces_geobrix.greenspace_site_{username}")
    .filter("function IN ('Play Space', 'Playing Field', 'Public Park Or Garden')")
)
df_greenspaces_bronze.createOrReplaceTempView("greenspace_site_bronze_vw")

greenspace_site_covered = (
    spark.sql("""
        SELECT 
            g1.id AS g1_id,
            g2.id AS g2_id,
            g1.function AS g1_function,
            g2.function AS g2_function,
            g2.distinctive_name_1 AS g2_name,
            g2.geometry AS geometry,
            ST_Geohash(ST_Transform(g2.geometry, 4326), 5) AS geohash
        FROM greenspace_site_bronze_vw g1
        INNER JOIN greenspace_site_bronze_vw g2
            ON ST_Covers(g2.geometry, g1.geometry)
            AND g1.id != g2.id
    """)
    .repartitionByRange(10, "geohash")
)
greenspace_site_covered.createOrReplaceTempView("greenspace_site_covered_vw")

greenspace_site_aggregated = (
    spark.sql("""
        WITH tmp AS (
            SELECT *, ST_AsEWKB(geometry) AS wkb_geometry
            FROM greenspace_site_covered_vw
        )
        SELECT 
            g2_id AS id,
            concat_ws(', ', any_value(g2_function), collect_set(g1_function)) AS functions,
            count(*) + 1 AS num_functions,
            g2_name AS name,
            ST_Area(ST_SetSrid(ST_GeomFromEWKB(wkb_geometry),27700)) AS area,
            ST_SetSrid(ST_GeomFromEWKB(wkb_geometry),27700) AS geometry,
            ST_Geohash(ST_Transform(ST_SetSrid(ST_GeomFromEWKB(wkb_geometry),27700), 4326), 5) AS geohash
        FROM tmp
        GROUP BY g2_id, g2_name, wkb_geometry
    """)
    .repartitionByRange(10, "geohash")
)
greenspace_site_aggregated.createOrReplaceTempView("greenspace_site_aggregated_vw")

greenspace_site_non_covered = (
    spark.sql("""
        SELECT 
            id, 
            function, 
            1 AS num_functions, 
            distinctive_name_1 AS name, 
            ST_Area(geometry) AS area, 
            geometry,
            ST_Geohash(ST_Transform(geometry, 4326), 5) AS geohash
        FROM greenspace_site_bronze_vw
        WHERE id NOT IN (SELECT g1_id FROM greenspace_site_covered_vw)
          AND id NOT IN (SELECT g2_id FROM greenspace_site_covered_vw)
    """)
    .repartitionByRange(10, "geohash")
)
greenspace_site_non_covered.createOrReplaceTempView("greenspace_site_non_covered_vw")

greenspace_site_all = (
    spark.sql("""
        SELECT * FROM greenspace_site_aggregated_vw
        UNION ALL
        SELECT * FROM greenspace_site_non_covered_vw
    """)
    .repartitionByRange(10, "geohash")
)

# Calculate 0%, 20%, 40%, 60%, 80%, 100% quantiles for area
quantiles = greenspace_site_all.approxQuantile("area", [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 0.001)
print("Quintile breakpoints:", quantiles)

q0, q20, q40, q60, q80, q100 = quantiles

greenspace_site_all = greenspace_site_all.withColumn(
    "area_category",
    F.when(F.col("area") <= q20, 20)
     .when(F.col("area") <= q40, 40)
     .when(F.col("area") <= q60, 60)
     .when(F.col("area") <= q80, 80)
     .otherwise(100)
)

display(greenspace_site_all.groupBy("area_category").count().orderBy("area_category"))
greenspace_site_all.createOrReplaceTempView("greenspace_site_all_vw")

In [0]:
# This block assigns each greenspace to the administrative boundary (county) with which it has the largest area of intersection.
# For each greenspace, it calculates the intersection area with all overlapping boundaries, ranks them, and selects the boundary with the maximum intersection.
# The result is a silver table mapping greenspaces to their most representative administrative boundary.

greenspace_site_silver = spark.sql("""
    WITH tmp AS (
        SELECT 
            a.id, 
            a.functions, 
            a.num_functions, 
            a.name, 
            a.area, 
            a.area_category, 
            a.geometry, 
            a.geohash,
            RANK() OVER (
                PARTITION BY a.id 
                ORDER BY ST_Area(ST_Intersection(a.geometry, b.geometry)) DESC
            ) AS administrative_rank,
            b.fid AS administrative_fid
        FROM greenspace_site_all_vw a
        INNER JOIN administrative_boundaries_vw b
            ON ST_Intersects(a.geometry, b.geometry)
    )
    SELECT 
        tmp.id, 
        tmp.functions, 
        tmp.num_functions, 
        tmp.name, 
        tmp.area, 
        tmp.area_category, 
        tmp.administrative_fid, 
        tmp.geometry, 
        tmp.geohash
    FROM tmp
    WHERE administrative_rank = 1
""").repartitionByRange(10, "geohash")

greenspace_site_silver.createOrReplaceTempView("greenspace_site_silver_vw")
greenspace_site_silver.write.mode("overwrite").option("mergeSchema", "true").saveAsTable(
    f"geospatial.greenspaces_geobrix.greenspace_site_silver_{username}"
)

In [0]:
# This block calculates the degree (number of connected links) for each road node,
# assigns a geohash for spatial partitioning, and writes the result as a silver table in Unity Catalog.

road_nodes_silver = (
    spark.sql(f"""
        WITH tmp AS (
            SELECT *, ST_AsEWKB(geometry) AS wkb_geometry
            FROM geospatial.networks_geobrix.road_node_{username}
            
        )
        SELECT 
            a.fid, 
            a.id, 
            a.form_of_road_node, 
            COUNT(DISTINCT b.id) AS degree, 
            ST_SetSrid(ST_GeomFromEWKB(a.wkb_geometry),27700) AS geometry,
            ST_Geohash(ST_Transform(ST_SetSrid(ST_GeomFromEWKB(a.wkb_geometry),27700), 4326), 5) AS geohash
        FROM geospatial.networks_geobrix.road_node_{username} a
        JOIN geospatial.networks_geobrix.road_link_{username} b
            ON a.id = b.start_node
            OR a.id = b.end_node
        GROUP BY a.fid, a.id, a.form_of_road_node, a.wkb_geometry
        ORDER BY COUNT(DISTINCT b.id) DESC
    """)
    .repartitionByRange(10, "geohash")
)

road_nodes_silver.write.mode("overwrite").saveAsTable(f"geospatial.networks_geobrix.road_node_silver_{username}")

In [0]:
# This block filters greenspace access points to include only those linked to silver greenspace sites (the attractive green spaces) and with pedestrian or mixed access types.
greenspace_entries = (
    spark.sql(f"""
        SELECT 
            a.fid, 
            a.id, 
            a.access_type, 
            a.ref_to_greenspace_site, 
            a.geometry, 
            ST_Geohash(ST_Transform(a.geometry, 4326), 5) AS geohash
        FROM geospatial.greenspaces_geobrix.access_point_{username} a
        WHERE a.ref_to_greenspace_site IN (SELECT id FROM greenspace_site_silver_vw)
          AND a.access_type IN ('Pedestrian', 'Motor Vehicle And Pedestrian')
    """)
    .repartitionByRange(10, "geohash")
)

greenspace_entries.createOrReplaceTempView("greenspace_entries_vw")

In [0]:
# This block finds the nearest road node for each greenspace entry point using a spatial kNN join (k=1),
# calculates the distance to the nearest road node, and writes the result as a silver table in Unity Catalog.
entry_road_1nn = (
    spark.sql(f"""
        WITH candidate_pairs AS (
            SELECT
                a.fid,
                a.id,
                a.access_type,
                a.ref_to_greenspace_site,
                a.geometry AS a_geometry,
                a.geohash,
                b.fid AS road_fid,
                b.geometry AS b_geometry,
                ST_Distance(a.geometry, b.geometry) AS dist
            FROM greenspace_entries_vw a
            JOIN geospatial.networks_geobrix.road_node_silver_{username} b
            ON substr(a.geohash, 1, 6) = substr(b.geohash, 1, 6)  -- spatial pruning!
        ),
        ranked AS (
            SELECT
                *,
                ROW_NUMBER() OVER (
                    PARTITION BY fid
                    ORDER BY dist ASC
                ) AS rn
            FROM candidate_pairs
        )
        SELECT
            fid,
            id,
            access_type,
            ref_to_greenspace_site,
            road_fid AS nearest_road_node_fid,
            dist AS distance_to_road_node,
            a_geometry AS geometry,
            geohash
        FROM ranked
        WHERE rn = 1
    """)
    .repartitionByRange(10, "geohash")
)

entry_road_1nn.write.mode("overwrite").saveAsTable(
    f"geospatial.greenspaces_geobrix.access_point_silver_{username}"
)